<a href="https://colab.research.google.com/github/SeongwonTak/TIL_swtak/blob/master/210119LogisticRegression1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 210119 로지스틱 회귀 분석 1

로지스틱 회귀 분석은 분명히 모델은 "회귀" 모델인데, "분류"문제를 푸는 신기한 모델이다. 무엇을 하려는 모델인지 확인 후, 실제 예시와 함께 간단한 수학적 배경을 알아보고자 한다.

## 1. 로지스틱 회귀란?
로지스틱 회귀는 분명히 회귀 모델인데 그 결과가 범주형일때 사용한다. 기본적으로는 "이진 분류"를 위한 선형 모델이지만 다중 분류로도 확장이 가능하다.

"이진 분류"를 위해 특정 샘플에 속할 확률을 돌려주고 이를 바탕으로 분류를 하는 모델을 생각할 것이다.

우선 코드 샘플을 통해 어떻게 쓸 것이고, 어떤 결과를 얻는지부터 확인해 보자.


##2, 로지스틱 회귀의 예시
예시의 데이터로, 미국 의대생의 입학 데이터를 받아오고자 한다.



In [2]:
import statsmodels.api as sm
import pandas as pd
import numpy as np

data_med = sm.datasets.get_rdataset("MedGPA", package="Stat2Data")
df_med = data_med.data
df_med.tail()

,Accept,Acceptance,Sex,BCPM,GPA,VR,PS,WS,BS,MCAT,Apps
50,D,0,M,2.41,2.72,8,8,8.0,8,32,7
51,D,0,M,3.51,3.56,11,8,6.0,9,34,6
52,A,1,F,3.43,3.48,7,10,7.0,10,34,14
53,D,0,M,2.61,2.80,7,5,NaN,6,18,6
54,D,0,M,3.36,3.44,11,11,8.0,9,39,1


데이터를 보면, 
- Acceptance : 0은 불합격, 1은 합격
- BCPM : 이과계열 과목 평균
- GPA : 학점
- VR : 영어
- PS : 물리
- WS : 쓰기
- BS : 생물
- MCAT : 시험점수합계.. 필요 없다,
- Apps : 지원횟수



In [9]:
# 결측값이 있는거 같다.
df_med.isna().sum()

Accept        0
Acceptance    0
Sex           0
BCPM          0
GPA           0
VR            0
PS            0
WS            1
BS            0
MCAT          0
Apps          0
dtype: int64

In [10]:
df_med['WS'].mean()

7.148148148148148

In [15]:
df_med['WS'] = df_med['WS'].fillna(7.1)

In [23]:
import statsmodels.api as sm
model_med = sm.Logit.from_formula("Acceptance ~ Sex + BCPM + GPA + VR + PS + WS + BS + Apps", df_med)
result_med = model_med.fit()
print(result_med.summary())

Optimization terminated successfully.
         Current function value: 0.275631
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:             Acceptance   No. Observations:                   55
Model:                          Logit   Df Residuals:                       46
Method:                           MLE   Df Model:                            8
Date:                Tue, 19 Jan 2021   Pseudo R-squ.:                  0.6000
Time:                        14:32:45   Log-Likelihood:                -15.160
converged:                       True   LL-Null:                       -37.896
Covariance Type:            nonrobust   LLR p-value:                 2.995e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -46.6414     15.600     -2.990      0.003     -77.216     -16.067
Sex[T.M]      -2.2835      1.

In [ ]:
P>|Z| 부분에서 0.05 이하여야, 유효한 계수이다.
따라서, 유효한 부분만을 살려서 다시 분석할 것이다.

In [24]:
model_med = sm.Logit.from_formula("Acceptance ~ PS + BS", df_med)
result_med = model_med.fit()
print(result_med.summary())

Optimization terminated successfully.
         Current function value: 0.460609
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:             Acceptance   No. Observations:                   55
Model:                          Logit   Df Residuals:                       52
Method:                           MLE   Df Model:                            2
Date:                Tue, 19 Jan 2021   Pseudo R-squ.:                  0.3315
Time:                        14:35:46   Log-Likelihood:                -25.333
converged:                       True   LL-Null:                       -37.896
Covariance Type:            nonrobust   LLR p-value:                 3.503e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -15.5427      4.684     -3.318      0.001     -24.723      -6.362
PS             0.4798      0.

위의 모델의 의미를 정리하면
$$ 0.4798PS + 1.1464BS - 15.5427$$

이 값이 0.5보다 클 경우 양성 피드백을 가진다.
즉, 저 값이 0.5보다 클 경우 합격으로 분류된다.
아닐 경우, 불합격으로 분류된다.

## 3. 로지스틱 회귀 분석의 이해

위에서 양성 샘플에 대해서 이야기 했는데 이에 대해 조금 더 언급하고자 한다.

로지스틱 회귀 분석의 목표는 어떤 샘플이 특정 클래스에 속할 확률을 예측하는 것이 목표이다.

* Odds ratio (오즈비) = $ \frac{P}{1-P}$,  $P$는 양성 샘플일 확률

여기의 양변에 로그를 취해 **logit 함수를 정의하게 된다.**

즉, 가중치 벡터 $w$가 주어질 때,
$$ logit(P(y=1|x))=w^{T}x$$
즉, 선형결합의 형태로 주어진다.


확률을 예측하기 위해, **logistic sigmoid function**을 사용한다. 굳이 이 함수를 사용하는 이유에 대해서는 추후에 언급하고자 한다. (정확히는 활성함수라는 개념에 대해 선행되어야 한다고 하는데, 조금 더 찾아봐야 한다)


추후에는 로지스틱 회귀에서 어떤 함수를 최척화 하는지, 시그모이드 함수를 왜 사용하는지에 대해 찾아보고 다른 예제와 다중 class로의 확장에 대해 알아보고자 한다.